## Objective

Build draft metabolic models for the 44 organisms with carbon source data (from CDMSCI-196) using RAST-annotated genomes and the complete ModelSEEDpy protocol:

1. Build base model with GramNegModelTemplateV6
2. **ATP correction** with Core-V5.2 template
3. Genome-scale gap-filling with GramNegModelTemplateV6

Note: While 57 organisms were annotated with RAST, only 44 organisms have carbon source growth data after filtering in CDMSCI-196. We only build models for these 44 organisms.


## Setup

In [ ]:
import pickle
import json
import pandas as pd
from pathlib import Path
from cobra.io import save_json_model, load_json_model

# ModelSEEDpy imports
from modelseedpy import MSBuilder, MSMedia, MSGapfill, MSATPCorrection
from modelseedpy.core.mstemplate import MSTemplateBuilder
from modelseedpy.core.msmodel import get_reaction_constraints_from_direction
from modelseedpy.core.msatpcorrection import load_default_medias

print("Imports successful")

## Configuration

In [ ]:
# Paths
GENOME_DIR = Path('results/genomes')
CORE_TEMPLATE_PATH = Path('../references/build_metabolic_model/Core-V5.2.json')
GRAMNEG_TEMPLATE_PATH = Path('../references/build_metabolic_model/GramNegModelTemplateV6.json')
MODEL_DIR = Path('models')
RESULTS_DIR = Path('results')

# Create output directories
MODEL_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Output files
STATS_FILE = RESULTS_DIR / 'model_statistics.csv'
GAPFILL_REPORT = RESULTS_DIR / 'gapfill_report.csv'

print(f"Configuration set")
print(f"  Genomes: {GENOME_DIR}")
print(f"  Core template: {CORE_TEMPLATE_PATH}")
print(f"  GramNeg template: {GRAMNEG_TEMPLATE_PATH}")
print(f"  Models: {MODEL_DIR}")
print(f"  Statistics: {STATS_FILE}")

## Load Template

In [ ]:
print("Loading ModelSEED templates...")
print()

# Load Core-V5.2 (for ATP correction)
with open(CORE_TEMPLATE_PATH) as fh:
    template_core = MSTemplateBuilder.from_dict(json.load(fh)).build()
print(f"Core-V5.2 template loaded:")
print(f"  Reactions: {len(template_core.reactions):,}")
print(f"  Compounds: {len(template_core.compcompounds):,}")
print()

# Load GramNegModelTemplateV6 (for base model and genome-scale gapfilling)
with open(GRAMNEG_TEMPLATE_PATH) as fh:
    template_gramneg = MSTemplateBuilder.from_dict(json.load(fh)).build()
print(f"GramNegModelTemplateV6 template loaded:")
print(f"  Reactions: {len(template_gramneg.reactions):,}")
print(f"  Compounds: {len(template_gramneg.compcompounds):,}")


## Load Default ATP Correction Medias

In [ ]:
# Load 54 default medias for ATP correction
default_medias = load_default_medias()
print(f"Loaded {len(default_medias)} default medias for ATP correction")
print(f"\nFirst 5 medias:")
for i, (media, min_obj) in enumerate(default_medias[:5], 1):
    print(f"  {i}. {media.id} (min_obj: {min_obj})")


## Define Pyruvate Minimal Media

In [ ]:
# Pyruvate minimal media composition
pyruvate_media_dict = {
    'cpd00020': (-5, 100),     # Pyruvate (carbon source)
    'cpd00007': (-10, 100),    # O2
    'cpd00001': (-100, 100),   # H2O
    'cpd00009': (-100, 100),   # Phosphate
    'cpd00013': (-100, 100),   # NH3 (nitrogen source)
    'cpd00048': (-100, 100),   # Sulfate
    'cpd00099': (-100, 100),   # Cl-
    'cpd00067': (-100, 100),   # H+
    'cpd00205': (-100, 100),   # K+
    'cpd00254': (-100, 100),   # Mg2+
    'cpd00971': (-100, 100),   # Na+
    'cpd00149': (-100, 100),   # Co2+
    'cpd00063': (-100, 100),   # Ca2+
    'cpd00058': (-100, 100),   # Cu2+
    'cpd00034': (-100, 100),   # Zn2+
    'cpd00030': (-100, 100),   # Mn2+
    'cpd10515': (-100, 100),   # Fe2+
    'cpd10516': (-100, 100),   # Fe3+
    'cpd11574': (-100, 100),   # Molybdate
    'cpd00244': (-100, 100),   # Ni2+
}

pyruvate_media = MSMedia.from_dict(pyruvate_media_dict)

print("Pyruvate minimal media defined")
print(f"  Total compounds: {len(pyruvate_media_dict)}")
print(f"  Carbon source: Pyruvate (cpd00020) at -5 mmol/gDW/hr")

## Load Filtered Organism List

Load the 44 organisms with carbon source data from CDMSCI-196 and map to orgIds for genome file lookup.

In [ ]:
# Load filtered organism list from CDMSCI-196
filtered_matrix = pd.read_csv('../CDMSCI-196-carbon-sources/results/combined_growth_matrix_filtered.csv', index_col=0, nrows=0)
filtered_organism_names = filtered_matrix.columns.tolist()

# Load organism metadata to map Species_Name to orgId
metadata = pd.read_csv('../CDMSCI-196-carbon-sources/results/organism_metadata.csv')

# Map filtered organism names to orgIds
filtered_orgids = []
for name in filtered_organism_names:
    match = metadata[metadata['Species_Name'] == name]
    if len(match) > 0:
        filtered_orgids.append(match.iloc[0]['orgId'])
    else:
        print(f"WARNING: Could not find orgId for '{name}'")

print(f"Filtered organism list loaded from CDMSCI-196:")
print(f"  Total organisms with carbon source data: {len(filtered_orgids)}")
print(f"  Excluded organisms without carbon source data: {57 - len(filtered_orgids)}")

# Get all genome pickle files
all_genome_files = sorted(GENOME_DIR.glob('*_genome.pkl'))

# Filter to only organisms with carbon source data
genome_files = [gf for gf in all_genome_files 
                if gf.stem.replace('_genome', '') in filtered_orgids]

print(f"\nGenome files:")
print(f"  Total available: {len(all_genome_files)}")
print(f"  Filtered to organisms with carbon source data: {len(genome_files)}")

print(f"\nFirst 5 organisms to process:")
for i, gf in enumerate(genome_files[:5], 1):
    print(f"  {i}. {gf.stem.replace('_genome', '')}")

## Build and Gap-Fill Models

Following the complete ModelSEEDpy protocol:

For each organism:
1. Load RAST-annotated genome
2. Build base model using MSBuilder.build_base_model()
3. Add ATP maintenance reaction (ATPM)
4. **ATP correction** with Core-V5.2 template (54 default medias)
5. Test biomass production on pyruvate media
6. Gap-fill using MSGapfill with GramNegModelTemplateV6
7. Integrate gapfill solution into model
8. Test gap-filled model
9. Save both draft and gap-filled models as JSON
10. Collect statistics


In [ ]:
# Helper function to integrate gapfill solution (from reference notebook)
def integrate_gapfill_solution(template_gramneg, model, gapfill_result):
    """
    Integrate gapfill solution by adding reactions from template to model.
    Returns list of added reactions.
    """
    added_reactions = []
    
    # Process new reactions
    gap_sol = {}
    for rxn_id, direction in gapfill_result.get('new', {}).items():
        # Skip exchange reactions (EX_*) - they'll be added automatically
        if rxn_id.startswith('EX_'):
            continue
            
        # Remove index suffix (e.g., rxn05481_c0 -> rxn05481_c)
        # Template reactions have format: rxn#####_c or rxn#####_e
        # Gapfill returns: rxn#####_c0 or rxn#####_e0
        if rxn_id.endswith('0'):
            template_rxn_id = rxn_id[:-1]  # Remove just the 0
        else:
            template_rxn_id = rxn_id
            
        if template_rxn_id in template_gramneg.reactions:
            gap_sol[template_rxn_id] = get_reaction_constraints_from_direction(direction)
    
    # Add reactions to model
    for rxn_id, (lb, ub) in gap_sol.items():
        template_reaction = template_gramneg.reactions.get_by_id(rxn_id)
        model_reaction = template_reaction.to_reaction(model)
        model_reaction.lower_bound = lb
        model_reaction.upper_bound = ub
        added_reactions.append(model_reaction)
    
    model.add_reactions(added_reactions)
    
    # Add missing exchanges
    add_exchanges = MSBuilder.add_exchanges_to_model(model)
    
    return added_reactions, add_exchanges

# Helper function to set media on model
def apply_media_to_model(media, model, prefix='EX_'):
    """Apply media constraints to model medium."""
    import math
    medium = {}
    for cpd, (lb, ub) in media.get_media_constraints().items():
        rxn_exchange = f'{prefix}{cpd}'
        if rxn_exchange in model.reactions:
            medium[rxn_exchange] = math.fabs(lb)
    return medium

print("Helper functions defined")

In [ ]:
# Test on first organism (ANA3)
test_genome_file = genome_files[0]
organism_id = test_genome_file.stem.replace('_genome', '')

print(f"Testing workflow on: {organism_id}")
print("="*60)

# Load genome
with open(test_genome_file, 'rb') as f:
    test_genome = pickle.load(f)
print(f"1. Loaded genome: {len(test_genome.features)} features")

# Build base model
builder = MSBuilder(test_genome, template_gramneg, organism_id)
test_model = builder.build_base_model(organism_id, annotate_with_rast=False)
builder.add_atpm(test_model)
print(f"2. Built model: {len(test_model.reactions)} reactions, {len(test_model.genes)} genes")

# Test on pyruvate media
test_model.medium = apply_media_to_model(pyruvate_media, test_model)
test_model.objective = 'bio1'
draft_sol = test_model.optimize()
print(f"3. Draft growth: {draft_sol.objective_value:.6f}")

# Run gapfilling
gapfiller = MSGapfill(test_model, default_gapfill_templates=[template_gramneg], default_target='bio1')
gapfill_result = gapfiller.run_gapfilling(pyruvate_media)

print(f"\n4. Gapfill result:")
print(f"   New reactions: {len(gapfill_result.get('new', {}))}")
print(f"   Reversed reactions: {len(gapfill_result.get('reversed', {}))}")

# Show first few gapfill reactions
print(f"\n5. First 5 gapfill reactions:")
for i, (rxn_id, direction) in enumerate(list(gapfill_result.get('new', {}).items())[:5], 1):
    print(f"   {i}. {rxn_id}: {direction}")
    
    # Check what happens when we try to look it up in template
    if rxn_id.endswith('_c0') or rxn_id.endswith('_e0'):
        template_rxn_id = rxn_id[:-3]  # Remove _c0 or _e0
        template_rxn_id_alt = rxn_id[:-1]  # Remove just the 0
    else:
        template_rxn_id = rxn_id
        template_rxn_id_alt = rxn_id
    
    in_template_1 = template_rxn_id in template_gramneg.reactions
    in_template_2 = template_rxn_id_alt in template_gramneg.reactions
    
    print(f"      Without suffix (_c0/_e0): '{template_rxn_id}' in template? {in_template_1}")
    print(f"      Without index (just 0): '{template_rxn_id_alt}' in template? {in_template_2}")

print(f"\n6. Checking template reaction format:")
# Get a few reaction IDs from template to see their format
for i, rxn in enumerate(list(template_gramneg.reactions)[:5], 1):
    print(f"   {i}. {rxn.id}")

print("\n" + "="*60)

In [ ]:
# Test integration on ANA3
print(f"Testing gapfill integration on: {organism_id}")
print("="*60)

# Copy the model
test_model_gapfilled = test_model.copy()

# Integrate gapfill solution
added_rxns, added_exch = integrate_gapfill_solution(template_gramneg, test_model_gapfilled, gapfill_result)

print(f"Integration results:")
print(f"  Added reactions: {len(added_rxns)}")
print(f"  Added exchanges: {len(added_exch)}")

# List the added reactions
print(f"\nAdded reactions:")
for i, rxn in enumerate(added_rxns[:10], 1):
    print(f"  {i}. {rxn.id}: {rxn.reaction}")

# Test gap-filled model
test_model_gapfilled.medium = apply_media_to_model(pyruvate_media, test_model_gapfilled)
test_model_gapfilled.objective = 'bio1'
gapfilled_sol = test_model_gapfilled.optimize()

print(f"\nGap-filled model growth: {gapfilled_sol.objective_value:.6f}")
print(f"Growth improved: {gapfilled_sol.objective_value > draft_sol.objective_value}")

print("="*60)

In [ ]:
model_stats = []
gapfill_results = []

print("="*80)
print("BUILDING AND GAP-FILLING MODELS")
print("="*80)
print(f"\nProcessing {len(genome_files)} organisms...\n")

for i, genome_file in enumerate(genome_files, 1):
    organism_id = genome_file.stem.replace('_genome', '')
    
    print(f"\n[{i}/{len(genome_files)}] {organism_id}")
    print("-" * 60)
    
    try:
        # 1. Load genome
        with open(genome_file, 'rb') as f:
            genome = pickle.load(f)
        print(f"  Loaded genome: {len(genome.features)} features")
        
        # 2. Build base model (following reference workflow)
        print(f"  Building base model...")
        builder = MSBuilder(genome, template_gramneg, organism_id)
        model_base = builder.build_base_model(organism_id, annotate_with_rast=False)
        
        # 3. Add ATPM reaction
        
        # 4. ATP Correction with Core-V5.2
        print(f"  ATP correction with Core-V5.2...")
        atp_correction = MSATPCorrection(
            model_base,
            template_core,
            default_medias,
            compartment='c0',
            atp_hydrolysis_id='ATPM_c0',
            load_default_medias=False
        )
        
        media_eval = atp_correction.evaluate_growth_media()
        atp_correction.determine_growth_media()
        atp_correction.apply_growth_media_gapfilling()
        atp_correction.expand_model_to_genome_scale()
        tests = atp_correction.build_tests()
        
        atp_corrected_reactions = len(model_base.reactions)
        atp_reactions_added = atp_corrected_reactions - draft_reactions
        print(f"    Added {atp_reactions_added} reactions during ATP correction")
        
        builder.add_atpm(model_base)
        
        draft_reactions = len(model_base.reactions)
        draft_metabolites = len(model_base.metabolites)
        draft_genes = len(model_base.genes)
        
        print(f"    Reactions: {draft_reactions}")
        print(f"    Metabolites: {draft_metabolites}")
        print(f"    Genes: {draft_genes}")
        
        # 5. Test draft model on pyruvate media
        model_base.medium = apply_media_to_model(pyruvate_media, model_base)
        model_base.objective = 'bio1'
        draft_solution = model_base.optimize()
        draft_growth = draft_solution.objective_value
        
        print(f"  Draft model growth: {draft_growth:.4f}")
        
        # Save draft model
        draft_file = MODEL_DIR / f"{organism_id}_draft.json"
        save_json_model(model_base, str(draft_file))
        print(f"  Saved draft: {draft_file.name}")
        
        # 6. Gap-fill for biomass production
        if draft_growth < 0.001:
            print(f"  Gap-filling...")
            
            # Create MSGapfill object (following reference workflow)
            gapfiller = MSGapfill(
                model_base,
                default_gapfill_templates=[template_gramneg],
                default_target='bio1'
            )
            
            # Run gapfilling
            gapfill_result = gapfiller.run_gapfilling(pyruvate_media)
            
            num_gapfilled = len(gapfill_result.get('new', {}))
            print(f"    Found {num_gapfilled} reactions to add")
            
        # 7. Integrate gapfill solution into model
            if num_gapfilled > 0:
                model_gapfilled = model_base.copy()
                added_rxns, added_exch = integrate_gapfill_solution(template_gramneg, model_gapfilled, gapfill_result)
                print(f"    Integrated {len(added_rxns)} reactions, {len(added_exch)} exchanges")
                
        # 8. Test gap-filled model
                model_gapfilled.medium = apply_media_to_model(pyruvate_media, model_gapfilled)
                model_gapfilled.objective = 'bio1'
                gapfilled_solution = model_gapfilled.optimize()
                gapfilled_growth = gapfilled_solution.objective_value
                
                print(f"  Gap-filled growth: {gapfilled_growth:.4f}")
            else:
                model_gapfilled = model_base.copy()
                gapfilled_growth = draft_growth
                print(f"  No gapfill solution found")
        else:
            print(f"  No gap-filling needed (draft grows)")
            model_gapfilled = model_base.copy()
            num_gapfilled = 0
            gapfilled_growth = draft_growth
        
        # 9. Save gap-filled model
        gapfilled_file = MODEL_DIR / f"{organism_id}_gapfilled.json"
        save_json_model(model_gapfilled, str(gapfilled_file))
        print(f"  Saved gap-filled: {gapfilled_file.name}")
        
        # 10. Collect statistics
        model_stats.append({
            'Organism_ID': organism_id,
            'Draft_Reactions': draft_reactions,
            'Draft_Metabolites': draft_metabolites,
            'Draft_Genes': draft_genes,
            'Draft_Growth': draft_growth,
            'ATP_Reactions_Added': atp_reactions_added,
            'Gapfilled_Reactions_Added': num_gapfilled,
            'Gapfilled_Growth': gapfilled_growth,
            'Status': 'Success'
        })
        
        gapfill_results.append({
            'Organism_ID': organism_id,
            'Gap_Filling_Needed': draft_growth < 0.001,
            'Reactions_Added': num_gapfilled,
            'Draft_Growth': draft_growth,
            'ATP_Reactions_Added': atp_reactions_added,
            'Gapfilled_Growth': gapfilled_growth
        })
        
    except Exception as e:
        print(f"  ERROR: {e}")
        import traceback
        traceback.print_exc()
        
        model_stats.append({
            'Organism_ID': organism_id,
            'Draft_Reactions': 0,
            'Draft_Metabolites': 0,
            'Draft_Genes': 0,
            'Draft_Growth': 0,
            'Gapfilled_Reactions_Added': 0,
            'Gapfilled_Growth': 0,
            'Status': f'Failed: {str(e)[:50]}'
        })

print(f"\n{'='*80}")
print("MODEL BUILDING COMPLETE")
print(f"{'='*80}")

## Save Statistics

In [ ]:
# Save model statistics
stats_df = pd.DataFrame(model_stats)
stats_df.to_csv(STATS_FILE, index=False)
print(f"Saved model statistics: {STATS_FILE}")

# Save gap-fill report
gapfill_df = pd.DataFrame(gapfill_results)
gapfill_df.to_csv(GAPFILL_REPORT, index=False)
print(f"Saved gap-fill report: {GAPFILL_REPORT}")

# Display first 10 rows
print(f"\nFirst 10 model statistics:")
display(stats_df.head(10))

## Summary

**Outputs Created:**
1. Draft models: `models/*_draft.json` (44 files, COBRApy JSON format)
2. Gap-filled models: `models/*_gapfilled.json` (44 files, COBRApy JSON format)
3. Model statistics: `results/model_statistics.csv`
4. Gap-fill report: `results/gapfill_report.csv`
5. Interactive viewer: `results/model_statistics_viewer.html`

**Model Format:**
- JSON format enables easier data extraction and programmatic access
- Compatible with all COBRApy functionality including FBA simulations
- Can be loaded with `cobra.io.load_json_model()` in CDMSCI-199

**Interactive Viewer:**
- Visualizes model building statistics with interactive plots
- Shows draft model size distributions
- Analyzes gap-filling impact and growth improvements
- Per-organism detailed results

**Next Steps:**
1. Review interactive viewer for overall model quality
2. Review gap-fill report for models that didn't grow
3. Proceed to CDMSCI-199: Test models with all carbon sources from CDMSCI-197

In [ ]:
# Run the viewer generation script
import subprocess
result = subprocess.run(['/Users/jplfaria/miniconda3/bin/python', 'create_model_stats_viewer.py'], 
                       capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

# Open in browser
viewer_path = Path('results/model_statistics_viewer.html').absolute()
print(f"\nViewer created at: {viewer_path}")
print(f"\nTo open in browser, run:")
print(f"open {viewer_path}")

## Generate Interactive Viewer

Create an interactive HTML page to visualize all model building statistics.

In [ ]:
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)

successful = stats_df[stats_df['Status'] == 'Success']
failed = stats_df[stats_df['Status'] != 'Success']

print(f"\nTotal organisms: {len(stats_df)}")
print(f"  Successful: {len(successful)}")
print(f"  Failed: {len(failed)}")

if len(successful) > 0:
    print(f"\nDraft Model Statistics (successful):")
    print(f"  Reactions: {successful['Draft_Reactions'].mean():.0f} ± {successful['Draft_Reactions'].std():.0f}")
    print(f"  Metabolites: {successful['Draft_Metabolites'].mean():.0f} ± {successful['Draft_Metabolites'].std():.0f}")
    print(f"  Genes: {successful['Draft_Genes'].mean():.0f} ± {successful['Draft_Genes'].std():.0f}")
    
    needed_gapfill = successful[successful['Gapfilled_Reactions_Added'] > 0]
    print(f"\nGap-filling Statistics:")
    print(f"  Models needing gap-filling: {len(needed_gapfill)}")
    print(f"  Average reactions added: {successful['Gapfilled_Reactions_Added'].mean():.0f}")
    
    growing = successful[successful['Gapfilled_Growth'] > 0.001]
    print(f"\nGrowth Results:")
    print(f"  Models growing after gap-fill: {len(growing)} ({100*len(growing)/len(successful):.1f}%)")
    print(f"  Average growth rate: {growing['Gapfilled_Growth'].mean():.4f}")

print(f"\n{'='*80}")

## Summary

**Outputs Created:**
1. Draft models: `models/*_draft.json` (44 files, COBRApy JSON format)
2. Gap-filled models: `models/*_gapfilled.json` (44 files, COBRApy JSON format)
3. Model statistics: `results/model_statistics.csv`
4. Gap-fill report: `results/gapfill_report.csv`

**Model Format:**
- JSON format enables easier data extraction and programmatic access
- Compatible with all COBRApy functionality including FBA simulations
- Can be loaded with `cobra.io.load_json_model()` in CDMSCI-199

**Next Steps:**
1. Review gap-fill report for models that didn't grow
2. Proceed to CDMSCI-199: Test models with all carbon sources from CDMSCI-197